In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive


Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
%cd ./drive/MyDrive/TB2021/
!ls

/content/drive/MyDrive/TB2021
0615_all		label_map.gsheet
0615_alltrain		labels_map.txt
0615_alltrain_label	__MACOSX
0615_allval		models
0615_allval_label	predict_0615_model_300epoch.csv
0616_all		predict_0615_model_500epoch.csv
0616_alltrain		predict_test.csv
0616_alltrain_label	Ranger-Deep-Learning-Optimizer
0616_allval		result
0616_allval_label	splited_clean_data
DataAugmentation.ipynb	splited_clean.zip
data_dic.txt		train.csv
day1.zip		train.gsheet
day2.zip		val.csv
label_map.csv		val.gsheet


In [ ]:
label_dict = {}
with open('./data_dic.txt','r') as d:
    txt = d.readlines()
    word_list = [word.split('\n')[0] for word in txt]
    for idx in range(len(word_list)):
        label_dict[word_list[idx]] = idx
    label_dict['l'] = 800
label_dict


{'l': 800,
 '丁': 502,
 '三': 573,
 '上': 240,
 '不': 433,
 '世': 322,
 '丞': 152,
 '中': 179,
 '主': 706,
 '久': 780,
 '之': 249,
 '事': 149,
 '于': 207,
 '五': 100,
 '亞': 309,
 '交': 379,
 '亨': 648,
 '京': 68,
 '亮': 38,
 '人': 759,
 '仁': 777,
 '介': 294,
 '仕': 331,
 '仙': 627,
 '代': 14,
 '仲': 298,
 '任': 577,
 '份': 291,
 '企': 384,
 '伊': 713,
 '伍': 363,
 '伯': 756,
 '伶': 536,
 '位': 136,
 '佑': 229,
 '何': 676,
 '余': 642,
 '作': 589,
 '佩': 113,
 '佳': 59,
 '來': 561,
 '侑': 686,
 '侯': 215,
 '俊': 605,
 '保': 367,
 '信': 320,
 '修': 784,
 '倉': 725,
 '倫': 226,
 '偉': 430,
 '健': 559,
 '傅': 747,
 '傑': 261,
 '備': 537,
 '傳': 727,
 '僑': 765,
 '儀': 305,
 '億': 546,
 '儒': 535,
 '優': 529,
 '允': 772,
 '元': 54,
 '兆': 42,
 '先': 111,
 '光': 398,
 '克': 578,
 '兒': 60,
 '內': 188,
 '全': 701,
 '公': 27,
 '其': 735,
 '具': 270,
 '典': 362,
 '冠': 132,
 '冷': 408,
 '凌': 635,
 '凍': 753,
 '凱': 221,
 '分': 447,
 '利': 44,
 '刷': 242,
 '券': 366,
 '創': 663,
 '劉': 110,
 '力': 176,
 '加': 542,
 '助': 661,
 '勁': 308,
 '勇': 552,
 '動': 560,
 '務': 319,
 '勝': 21

In [ ]:
!gdown --id '19DKJ9tFKkG7izZjpbVcJ5uBXJUc3e_bU' --output splited_clean.zip 

Downloading...
From: https://drive.google.com/uc?id=19DKJ9tFKkG7izZjpbVcJ5uBXJUc3e_bU
To: /content/drive/My Drive/TB2021/splited_clean.zip
146MB [00:00, 158MB/s]


In [ ]:
!gdown --id '1P699AMFAtlcLI_AgrrArgOPPXt0bynu1' --output day1.zip 

Downloading...
From: https://drive.google.com/uc?id=1P699AMFAtlcLI_AgrrArgOPPXt0bynu1
To: /content/drive/My Drive/TB2021/day1.zip
28.9MB [00:00, 92.4MB/s]


In [ ]:
!gdown --id '1ExcFvCZA8HhkUWLXUlDlqC_HEqqqVkeN' --output day2.zip 

Downloading...
From: https://drive.google.com/uc?id=1ExcFvCZA8HhkUWLXUlDlqC_HEqqqVkeN
To: /content/drive/My Drive/TB2021/day2.zip
17.3MB [00:00, 54.9MB/s]


In [ ]:
!unzip -O big5 day2.zip
!unzip -O big5 day1.zip
!unzip -O big5 splited_clean.zip

Streaming output truncated to the last 5000 lines.
  inflating: 0616_alltrain/1001_奕.jpg  
  inflating: 0616_alltrain/1002_N.jpg  
  inflating: 0616_alltrain/1003_興.jpg  
  inflating: 0616_alltrain/1004_游.jpg  
  inflating: 0616_alltrain/1005_瑞.jpg  
  inflating: 0616_alltrain/1006_行.jpg  
  inflating: 0616_alltrain/1007_重.jpg  
  inflating: 0616_alltrain/1008_婉.jpg  
  inflating: 0616_alltrain/1009_農.jpg  
  inflating: 0616_alltrain/100_益.jpg  
  inflating: 0616_alltrain/1010_寬.jpg  
  inflating: 0616_alltrain/1011_芸.jpg  
  inflating: 0616_alltrain/1012_盛.jpg  
  inflating: 0616_alltrain/1013_律.jpg  
  inflating: 0616_alltrain/1014_系.jpg  
  inflating: 0616_alltrain/1015_場.jpg  
  inflating: 0616_alltrain/1016_N.jpg  
  inflating: 0616_alltrain/1017_鍾.jpg  
  inflating: 0616_alltrain/1018_璟.jpg  
  inflating: 0616_alltrain/1019_宗.jpg  
  inflating: 0616_alltrain/101_N.jpg  
  inflating: 0616_alltrain/1020_材.jpg  
  inflating: 0616_alltrain/1021_憲.jpg  
  inflating: 0616_alltrain/1022

In [ ]:
%cd splited_clean_data

/content/drive/My Drive/TB2021/splited_clean_data


In [ ]:
%cd ..

/content/drive/My Drive/TB2021


## **Import Packages**


In [ ]:
# Import 
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import os
import cv2
from torch.utils.data import ConcatDataset, DataLoader
from torchvision.datasets import DatasetFolder
from tqdm.auto import tqdm

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [ ]:
# this function is for read image,the input is directory name
def read_directory(directory_name,label_dict):
    ## 重建方便DatasetFolder
    label_count = [0 for i in range(len(label_dict))]
    os.makedirs(directory_name + "_label",exist_ok=True)
    for word in label_dict:
        os.makedirs(directory_name + "_label/"+ str(label_dict[word]).zfill(3),exist_ok=True)
    check = 0
    filenames = []
    for filename in os.listdir(r"./"+directory_name):
        label = filename[-5]
        filenames.append(filename.split('_')[0])
        if label in label_dict: ##先不管沒出現的
            num_label = label_dict[label]
            img = cv2.imread(directory_name + "/" + filename)
            cv2.imwrite(directory_name + "_label/"+ str(num_label).zfill(3) + "/" + str(num_label) + "_" +str(label_count[num_label]) +".jpg",img)
            label_count[num_label] += 1
            check += 1
            if check % 500 == 0:
                print(check)
    return filenames

In [ ]:
%cd splited_clean_data/
!ls

/content/drive/My Drive/TB2021/splited_clean_data
test  test_kaggle  test_label  train  train_label  val	val_label


In [ ]:
train_name = read_directory("train",label_dict)
print(len(train_name))
print(train_name[:10])

500
1000
1500
2000
2500
3000
3500
4000
4500
['0', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008']


In [ ]:
test_name = read_directory("test",label_dict)
print(len(test_name))
print(test_name[:10])

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
6869
['32143_isnull.jpg', '52189_涵.jpg', '49675_好.jpg', '6580_健.jpg', '63107_銓.jpg', '47355_裕.jpg', '45926_東.jpg', '1088_祥.jpg', '17745_傳.jpg', '21128_石.jpg']


In [ ]:
val_name = read_directory("val",label_dict)
print(len(val_name))

501


In [ ]:
!ls

test  test_kaggle  test_label  train  train_label  val	val_label


In [ ]:
%cd ..

/content/drive/My Drive/TB2021


In [ ]:
train_name_0615 = read_directory("0615_alltrain",label_dict)
print(len(train_name_0615))
print(train_name_0615[:10])

In [ ]:
train_name_0616 = read_directory("0616_alltrain",label_dict)
print(len(train_name_0616))
print(train_name_0616[:10])

In [ ]:
val_name_0615 = read_directory("0615_allval",label_dict)
print(len(val_name_0615))
print(val_name_0615[:10])

In [ ]:
val_name_0616 = read_directory("0616_allval",label_dict)
print(len(val_name_0616))
print(val_name_0616[:10])

In [ ]:

train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [ ]:
batch_size = 128

# Construct datasets.
train_set = DatasetFolder("splited_clean_data/train_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
train_0615 = DatasetFolder("0615_alltrain_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
train_set_0616 = DatasetFolder("0616_alltrain_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("splited_clean_data/val_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("splited_clean_data/test_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_0615 = DatasetFolder("0615_allval_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set_0616 = DatasetFolder("0616_allval_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
mix_set = ConcatDataset([train_set, valid_set, test_set, train_0615, train_set_0616])
mix_valid = ConcatDataset([valid_0615, valid_set_0616])
# Construct data loaders.
train_loader = DataLoader(mix_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(mix_valid, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## **Model**
Base on reference, rewrite to pytorch version and add more dropout.

---


Reference: https://github.com/pavlo-melnyk/offline-HCCR 




In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.AvgPool2d(kernel_size, stride, padding)
        # torch.nn.AdaptiveAvgPool2d = keras' GlobalAvgPooling

        # input image size: [3, 96, 96]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),
            nn.Dropout(0.5),

            nn.Conv2d(64, 96, 3, 1, 1, bias = False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 64, 3, 1, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 96, 3, 1, 1, bias = False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),
            nn.Dropout(0.5),

            nn.Conv2d(96, 128, 3, 1, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 96, 3, 1, 1, bias = False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 128, 3, 1, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),
            nn.Dropout(0.5),

            nn.Conv2d(128, 256, 3, 1, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 192, 3, 1, 1, bias = False),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 256, 3, 1, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),
            nn.Dropout(0.5),

            nn.Conv2d(256, 448, 3, 1, 1, bias = False),
            nn.BatchNorm2d(448),
            nn.ReLU(),
            nn.Conv2d(448, 256, 3, 1, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 448, 3, 1, 1, bias = False),
            nn.BatchNorm2d(448),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),

            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.85)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(448, 801)
        )
    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 801]

        x = self.cnn_layers(x)
        x = x.flatten(1)
        x = self.fc_layers(x)
        return x


## **Training**



In [ ]:
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 14.0MB/s 
     |████████████████████████████████| 3.3MB 52.2MB/s 
     |████████████████████████████████| 901kB 46.2MB/s 


In [ ]:
!mkdir models
!ls

DataAugmentation.ipynb	Ranger-Deep-Learning-Optimizer	train.gsheet
data_dic.txt		splited_clean_data		val.csv
labels_map.txt		splited_clean.zip		val.gsheet
__MACOSX		train.csv
DataAugmentation.ipynb	models				train.csv
data_dic.txt		Ranger-Deep-Learning-Optimizer	train.gsheet
labels_map.txt		splited_clean_data		val.csv
__MACOSX		splited_clean.zip		val.gsheet


In [ ]:
## Class weight
weight = [0.0 for i in range(801)]

In [ ]:
for batch in tqdm(train_loader):
    img,labels = batch
    for label in labels:
        weight[label] += 1.0

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for idx in range(len(weight)):
    ori = weight[idx]
    if ori != 0:
        weight[idx] = sum(weight) / (801 * ori)
    else:
        weight[idx] = sum(weight) / (801)
weight

[1.0220557636287972,
 0.9423818357594957,
 0.6439952978250468,
 1.017749524854059,
 1.355475958166468,
 1.5237174872628332,
 0.9370503578092285,
 0.9358919068688472,
 0.7594708425508875,
 0.9332697386895924,
 0.932110924681976,
 0.80682506613717,
 0.67136981514109,
 0.9279248405377293,
 1.3386612969010838,
 0.5732565216489397,
 0.858063243015736,
 1.499559811239307,
 0.9221817756873829,
 0.5442402117812056,
 0.7964332294814971,
 0.8520547249375484,
 0.9163347488883709,
 1.1897266011140442,
 1.6980381311797055,
 0.9138190569149897,
 1.6949369821139117,
 0.7905290549891649,
 1.0763814691578488,
 0.7885204749245912,
 1.4762581190221424,
 1.180192045544683,
 0.6935829093675887,
 1.9617586474756183,
 1.6807871978625466,
 0.6532705372182799,
 0.6520674071404063,
 1.1715555485090536,
 1.0640485187953728,
 1.670304181848551,
 1.4606844338025395,
 1.9462185884811642,
 0.6866029782499049,
 0.8322774529879798,
 1.1635445237528332,
 0.8303152469933964,
 0.8929209156852916,
 1.159285694874735,
 0.8

In [ ]:
from transformers import get_linear_schedule_with_warmup
class_weight = torch.tensor(weight,dtype=torch.float)

device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
# Cross-Entropy with weight
criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
# optimizer SGD 
optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
# The number of training epochs.
n_epochs = 300
stop_count = 0
model_path = './models/HCCR_SGD_weight.ckpt'
total_step = len(train_loader) * n_epochs
## Linear scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,1,total_step)

best_acc = 0.0
step = 1

for epoch in range(n_epochs):
    model.train()
    # These are used to record information in training.
    train_loss = []
    train_accs = []
    # Iterate the training set by batches.
    for batch_idx, batch in enumerate(tqdm(train_loader)):
        imgs, labels = batch
        logits = model(imgs.to(device))
        loss = criterion(logits.to(device), labels.to(device))

        optimizer.zero_grad()
        loss.backward()

        # Clip the gradient norms for stable training.A
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()
        scheduler.step()
        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):
        imgs, labels = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    # Save Best Model
    if valid_acc > best_acc:
        torch.save(model.state_dict(),model_path)
        print('save model with acc:',valid_acc)
        best_acc = valid_acc

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



[ Train | 001/100 ] loss = 0.03724, acc = 0.94007



[ Valid | 001/100 ] loss = 0.12609, acc = 0.93239
save model with acc: tensor(0.9324, device='cuda:0')



[ Train | 002/100 ] loss = 0.03147, acc = 0.94203



[ Valid | 002/100 ] loss = 0.12333, acc = 0.92870


...


[ Valid | 098/100 ] loss = 0.12676, acc = 0.92922



[ Train | 099/100 ] loss = 0.02612, acc = 0.94528



[ Valid | 099/100 ] loss = 0.12627, acc = 0.92922



[ Train | 100/100 ] loss = 0.02590, acc = 0.94596



[ Valid | 100/100 ] loss = 0.12775, acc = 0.92974
